In [ ]:
from operator import itemgetter
list_bus_lines = [('150',), ('42',), ('130',), ('120',), ('14',),('155',),  ('54A',), ('16',), ('66',), ('37',), ('15A',), ('4',),('46A',),('145',), ('68A',), ('49',), ('40',), ('70',), ('38A',), ('83',), ('747',), ('27',), ('66B',), ('79',), ('122',), ('40D',), ('9',), ('11',), ('140',), ('1',), ('757',), ('47',), ('38',), ('25B',), ('15',), ('29A',), ('66A',), ('65',), ('7',), ('39A',), ('41',), ('15B',), ('77A',), ('83A',), ('27B',), ('25A',), ('40B',), ('151',), ('123',), ('32',), ('43',), ('27X',), ('84',), ('79A',), ('44',), ('39',), ('33X',), ('14C',), ('84A',), ('27A',), ('65B',), ('41C',), ('31A',), ('31',), ('13',), ('69',), ('7A',), ('40E',), ('25',), ('53',), ('61',), ('68',), ('26',), ('56A',), ('7D',), ('67',), ('33',), ('66X',), ('66E',), ('38B',), ('67X',), ('116',), ('16C',), ('44B',), ('38D',), ('31B',), ('41B',), ('25D',), ('32X',), ('51D',), ('16D',), ('46E',), ('84X',), ('7B',), ('69X',), ('15D',), ('41X',), ('142',), ('68X',), ('25X',), ('39X',), ('41D',), ('33D',), ('51X',), ('77X',), ('118',), ('33E',)]
list_bus_lines = list(map(itemgetter(0),list_bus_lines))
len(list_bus_lines)
#('150',), ('42',), ('130',), ('120',), ('14',),('155',),  ('54A',), ('16',), ('66',), ('37',), ('15A',), ('4',), 

In [ ]:
import pymysql,json
#Extract trip ids of individual bus lines
def extract_tripsid_bus_line(bus_line):
    print(bus_line)
    sql = "select distinct trips100.TRIPID,trips100.LINEID from dublinbus.trips100 where trips100.LINEID=%s and DAYOFSERVICE like %s"
    db = pymysql.connect(host="127.0.0.1",  # your host
                         port = 3307,
                         user="admin",  # username
                         password="Ucd-nl-2018",  # password
                         db="dublinbus", # name of the database
                        use_unicode=True, 
                         charset='utf8') 

    month = '%'+"DEC"+'%'
    cursor = db.cursor()
    cursor.execute(sql,(bus_line,month,))
    result = cursor.fetchall()
    return result

In [ ]:
import pymysql,json
def create_list_bus_lines(trip_id):

    sql = " select * from dublinbus.minLeavetimes where TRIPID =%s" 
    db = pymysql.connect(host="127.0.0.1",  # your host
                         port = 3307,
                         user="admin",  # username
                         password="Ucd-nl-2018",  # password
                         db="dublinbus", # name of the database
                        use_unicode=True, 
                         charset='utf8') 


    cursor = db.cursor()
    cursor.execute(sql,(trip_id,))
    result = cursor.fetchall()
    return result

In [ ]:
d = {} #Dictionary to store each bus-line's dataframe as an individual key

In [ ]:

def create_dataframe_individual_bus_lines(full_list_bus_line,bus_line):
    try:
        import numpy as np
        import pandas as pd
        d[bus_line] = pd.DataFrame()
        # Creating an empty Dataframe with column names only
        for i in full_list_bus_line:
            i = list(i)
            d[bus_line] = d[bus_line].append(i, ignore_index = True)
    except:
        print("There had been an error for ",bus_line)
        pass


In [ ]:
for i in list_bus_lines:
    result = extract_tripsid_bus_line(i)
    full_list_bus_line = []
    for j in result:
        get_data = create_list_bus_lines(j[0])
        full_list_bus_line.append(get_data)
    create_dataframe_individual_bus_lines(full_list_bus_line,i)
    

In [ ]:
#This keys are deleted beacuse there is no data related to them in UCD's database
del d['155']

del d['747']
del d['757']
del d['66E']

In [ ]:
list1 = list(d.keys())
print(list1)

In [ ]:
import numpy as np
import pandas as pd
import seaborn as sns
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn import metrics
pd.options.mode.chained_assignment=None
%matplotlib inline
import pickle
import datetime


#### This is for bus lines operating Monday-Sunday

In [ ]:
def train_models_generate_pickles(value):
    try:
        df = d[value].copy()
        df.drop(df.columns[[1,5,8]],axis=1,inplace=True)
        date = df[[0]]
        date = list(date[0])
        extract_date = []
        for i in date:
            new_date = i[0:9]
            extract_date.append(new_date)
        extract_week_day = []
        for i in extract_date:
            extract_week_day.append(datetime.datetime.strptime(i, '%d-%b-%y').strftime('%A'))
        df = df.assign(Extracted_Date=extract_date)
        df = df.assign(Day=extract_week_day)
        df.rename({0:'DateTime', 2: 'Sequence',3:'StopID',4:'Planned_Arr',6:'Actual_Arr',7:'Actual_Depart'},axis=1,inplace=True)  # new method
        data_input1= pd.DataFrame(df['Day'])
        dummy = pd.get_dummies(data_input1)
        df=pd.concat([df,dummy],axis=1)
        input_to_model=pd.DataFrame(df[['Sequence','StopID','Planned_Arr','Day_Monday','Day_Saturday','Day_Sunday','Day_Thursday','Day_Tuesday','Day_Wednesday']])
        #Day_Friday has been taken out because if mon-sun all are zeroes that means friday is 1. Using dummy one hot encoding concept
        #print(input_model)
        output = df['Actual_Arr']
        X_train,X_test,Y_train,Y_test=train_test_split(input_to_model,output,test_size=0.33,random_state=42)
        print("DataFrame is for ",value)
        print (X_train.shape)
        rf=RandomForestRegressor(n_estimators=20)
        rf.fit(X_train,Y_train)
        print(Y_test.shape)
        prediction=rf.predict(X_test)
        #Model Evaluation Parameters: RMSE
        print(np.sqrt(metrics.mean_squared_error(Y_test,prediction)))
        value = str(value)
        filename = value+".pickle"
        pickle.dump(rf,open(filename,'wb'))
    except:
        
        print("Error was for dataframe",value)
        pass
    
    
    

In [ ]:
for value in list1:
    train_models_generate_pickles(value)


#### This is for bus lines operating Monday-Friday only

In [ ]:
list2 = ['68A','747','66B','757','27X','33X','84A','7D','66X','66E','38B','67X','116','44B','38D','31B','25D', '32X', '51D', '16D', '46E', '84X', '7B', '69X', '15D', '41X', '142', '68X', '25X', '39X', '41D', '33D', '51X', '77X', '118', '33E']

In [ ]:

#fixes the "['Day_Sunday', 'Day_Saturday'] not in index" error beacuse these bus lines would not have dummy encoding for the weekends
def train_models_generate_pickles_mon_fri(value):
    try:
        df = d[value].copy()
        df.drop(df.columns[[1,5,8]],axis=1,inplace=True)
        date = df[[0]]
        date = list(date[0])
        extract_date = []
        for i in date:
            new_date = i[0:9]
            extract_date.append(new_date)
        extract_week_day = []
        for i in extract_date:
            extract_week_day.append(datetime.datetime.strptime(i, '%d-%b-%y').strftime('%A'))
        df = df.assign(Extracted_Date=extract_date)
        df = df.assign(Day=extract_week_day)
        df.rename({0:'DateTime', 2: 'Sequence',3:'StopID',4:'Planned_Arr',6:'Actual_Arr',7:'Actual_Depart'},axis=1,inplace=True)  # new method
        data_input1= pd.DataFrame(df['Day'])
        dummy = pd.get_dummies(data_input1)
        df=pd.concat([df,dummy],axis=1)
        input_to_model=pd.DataFrame(df[['Sequence','StopID','Planned_Arr','Day_Friday','Day_Monday','Day_Thursday','Day_Tuesday','Day_Wednesday']])
        output = df['Actual_Arr']
        X_train,X_test,Y_train,Y_test=train_test_split(input_to_model,output,test_size=0.33,random_state=42)
        print("DataFrame is for ",value)
        print (X_train.shape)
        rf=RandomForestRegressor(n_estimators=20)
        rf.fit(X_train,Y_train)
        print(Y_test.shape)
        prediction=rf.predict(X_test)
        #Model Evaluation Parameters: RMSE
        print(np.sqrt(metrics.mean_squared_error(Y_test,prediction)))
        value = str(value)
        filename = value+".pickle"
        pickle.dump(rf,open(filename,'wb'))
    except:
        
        print("Error was for dataframe",value)
        pass
    
    
    

In [ ]:
for value in list2:
    train_models_generate_pickles_mon_fri(value)
    